# debug web
#https://pythononpow.medium.com/debugging-a-tornado-pythononwheels-web-application-with-vscode-d9f5c5e5603f

In [4]:
# {
#     // 使用 IntelliSense 了解相关属性。 
#     // 悬停以查看现有属性的描述。
#     // 欲了解更多信息，请访问: https://go.microsoft.com/fwlink/?linkid=830387
#     "version": "0.2.0",
#     "configurations": [
        
#         {  
#             "name": "Python: PoW",  
#             "type": "python",  
#             "request": "launch",  
#             "program": "${workspaceFolder}/instock/web/web_service.py",  
#             "args": [],  
#             "console": "integratedTerminal"  
#         }
            
#     ]
# }

In [5]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime

import sys

# stdout=sys.stdout
sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd

# 初始化db

In [6]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}



## def func

In [7]:
def write_code_to_blk(codew, blk='060'):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
    if write_blk == 'y' or write_blk == 'Y':
        hdf5_wri = cct.cct_raw_input(
            "Rewrite code [Y] or append [N](defalut:N):")
        if hdf5_wri == 'y' or hdf5_wri == 'Y':
            append_status = False
        else:
            append_status = True
        if len(codew) > 3:
            cct.write_to_blocknew(block_path, codew, append_status,
                                  doubleFile=False, keep_last=0)
            print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write No:%s block_path:%s" % (len(codew), block_path))
            # ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

#allday
def groupby_count_code(df):
    df['couts'] = df.groupby('code')['code'].transform('count')
    if 'date' in df.columns:
        df = df.sort_values(by=['couts','date'],ascending=[0,1])
    else:
        df = df.sort_values(by=['couts'],ascending=[0])
    return df

In [8]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

#前一日
def day_last_days(daynow,last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

#上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result
    
# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0



In [9]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
        logging.error("执行信息：数据库不存在，将创建。")

## 放量上涨table 
cn_stock_strategy_entercn_stock_strategy_enter

In [21]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


def panda_df(conn,sql):
    df = pd.read_sql(sql, conn)
    print(df.shape)
    return df

from pandas import DataFrame as df
def exe_sql_select(conn,sql):
	cur = conn.cursor() 
	data = None
	try: # 使用异常处理，以防程序无法正常运行
		cur.execute(sql) 
		data = df(cur.fetchall(), columns = [col[0] for col in cur.description]) 
	except Exception as e:
		conn.rollback() # 发生错误时回滚
		print('事务处理失败', e)
	else:
		# conn.commit() # 事务提交
		print('事务处理成功', cur.rowcount)
	# cur.close()
	return data

def do_select_db(sql):
    conn = pymysql.connect(**MYSQL_CONN_DBAPI)
    mycursor = conn.cursor()
    print(sql)
    df = panda_df(conn,sql)
    conn.close()
    return df

### 读取SQL数据

In [22]:
conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcolall = f''' `date`,`code`,`name`,`rate_1` '''
_selcol = f''' `code`,`name`,`rate_1` '''

_table_name = 'cn_stock_strategy_enter'

lastday = last_tddate(1)

today_now = last_tddate(0)



sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{today_now}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' AND rate_1 > 0 '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' '''


sql_now_last7day = f'''SELECT {_selcolall} FROM `{_table_name}` WHERE `date` >= '{last_tddate(7)}' '''

sql_now_alltoday = f'''SELECT {_selcolall} FROM `{_table_name}`  '''

sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
print(sql_now_today)

mycursor = conn.cursor()
# mycursor.execute(sql_now_price)
# print(_selcol)
# count = 0
# for field in mycursor.fetchall():
# 		# print(field[0],end='  ')
# 		# print(field[1:4])
# 		# print(field)
# 		count +=1
# print("all:%s"%(count))


#lastday 
# df = exe_sql_select(conn,sql_now_price)

#today:
df = exe_sql_select(conn,sql_now_today)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn,sql_now_today)

# df3all = panda_df(conn,sql_now_alltoday)

dflast7d = panda_df(conn,sql_now_last7day)

# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
	
conn.close()

SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-12-27' AND rate_1 > 0 
SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-12-28' 
事务处理成功 16
(16, 3)
(149, 4)


In [12]:
#today
print(df2[:2],df2.shape)
vol_code = df2.code.tolist()[:2]
#allday
df2[:2]
dflast7d[:2]


     code  name  rate_1
0  000408  藏格矿业    2.55
1  002022  科华生物    6.57 (16, 3)


,date,code,name,rate_1
0,2023-12-19,000856,冀东装备,-4.08
1,2023-12-19,000892,欢瑞世纪,-3.90


In [13]:
# df3all = groupby_count_code(df3all)
dflast7d = groupby_count_code(dflast7d)

### show couts

In [14]:
filter_date = '2023-04-28'
filter_date = today_now
# last_tddate(0)
# filter_date = lastday

In [15]:
df7multiIndex = dflast7d.reset_index().set_index(['code','date'])
df7multiIndex[:5]
df7tail = df7multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])

### show Today

In [16]:
df7tail =  df7tail.reset_index().set_index('code')
# df7tail[ df7tail.rate_1 ] 
df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())][:6]

,date,index,name,rate_1,couts
code,,,,,
002962,2023-12-28,142,五方光电,10.00,3
002952,2023-12-28,141,亚世光电,9.99,4
002395,2023-12-28,136,双象股份,9.98,3
002660,2023-12-28,138,茂硕电源,9.97,2
300112,2023-12-28,144,万讯自控,8.77,2
002150,2023-12-28,135,通润装备,7.38,2


In [17]:
df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ][:10]

,date,index,name,rate_1,couts
code,,,,,
002962,2023-12-28,142,五方光电,10.00,3
605178,2023-12-28,148,时空科技,9.99,1
002952,2023-12-28,141,亚世光电,9.99,4
002395,2023-12-28,136,双象股份,9.98,3
002660,2023-12-28,138,茂硕电源,9.97,2
300112,2023-12-28,144,万讯自控,8.77,2
002150,2023-12-28,135,通润装备,7.38,2
002022,2023-12-28,134,科华生物,6.57,1
003028,2023-12-28,143,振邦智能,6.35,2


### 查询macd

In [25]:
# sql_select = '''select * FROM cn_stock_indicators WHERE code='000001' ORDER BY date desc limit 5'''
sql_select_code = f'''select date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,boll_ub,boll FROM cn_stock_indicators WHERE macd>0 AND macds >0 AND macdh >0 AND date=CURDATE() AND close >=boll_ub '''
df_macd_boll=do_select_db(sql_select_code)

select date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,boll_ub,boll FROM cn_stock_indicators WHERE macd>0 AND macds >0 AND macdh >0 AND date=CURDATE() AND close >=boll_ub 
(81, 12)


In [27]:
# df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])
# df7tail
df_macd_boll

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,boll_ub,boll
0,2023-12-28,000099,中信海直,8.55,0.061217,0.023891,0.037327,64.9098,52.2963,90.1368,8.42368,8.0865
1,2023-12-28,000400,许继电气,21.85,0.481680,0.470310,0.011371,78.6127,68.2976,99.2429,21.61380,20.8015
2,2023-12-28,000408,藏格矿业,25.29,0.315402,0.243570,0.071832,82.7065,74.7689,98.5816,25.03500,24.2485
3,2023-12-28,000532,华金资本,12.20,0.084339,0.079659,0.004680,44.7716,36.0853,62.1443,12.11760,11.6445
4,2023-12-28,000619,海螺新材,7.09,0.108492,0.058777,0.049714,52.5627,41.9825,73.7231,6.95340,6.5425
...,...,...,...,...,...,...,...,...,...,...,...,...
76,2023-12-28,605117,德业股份,83.99,3.286900,0.074733,3.212170,88.3801,80.9387,103.2630,82.25780,64.7115
77,2023-12-28,605118,力鼎光电,21.99,0.723473,0.421862,0.301611,78.0498,59.5428,115.0640,20.47460,18.0100
78,2023-12-28,605178,时空科技,25.21,0.418221,0.273537,0.144684,73.6920,57.1365,106.8030,24.54840,22.7880
79,2023-12-28,605196,华通线缆,7.72,0.031108,0.008106,0.023003,69.3379,63.1123,81.7890,7.67927,7.4655


#### 写入065 df_macd_boll 

In [28]:
write_code_to_blk(df_macd_boll.code.tolist(),'065')

write blk [Y] or [N]: y
Rewrite code [Y] or append [N](defalut:N): y


all write to D:\MacTools\WinTools\new_tdx2\T0002\blocknew\065.blk:81
inboll1 add:77
write:81 block_path:D:\MacTools\WinTools\new_tdx2\T0002\blocknew\065.blk


True

### 写入放量上涨061

In [54]:
# df7code = df7tail[df7tail.couts >1].index.get_level_values('code')
# filter_date = '2023-04-28' couts >1

df7code = df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())].index

#nowday
df7code = df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ].index
df7code
len(df7code)

Index(['002962', '605178', '002952', '002395', '002660', '300112', '002150',
       '002022', '003028', '603977', '301067', '002632', '002903', '300530',
       '000408', '002748'],
      dtype='object', name='code')

16

In [55]:

# hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
# if len(hdf5_wri) == 0:
#     pass
# elif hdf5_wri == 'y' or hdf5_wri == 'Y':
#     append_status = False
# else:
#     append_status = True
# df2.code.values
#today:
# write_code_to_blk(df7code,'061')

# write_code_to_blk(df7code,'062')

# write_code_to_blk(df7code,'064')


## 读取cn_stock_strategy_keep_increasing

In [56]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_strategy_keep_increasing'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = '2023-05-08'
start_d = last_tddate(5)
# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` > '{today_now}'- INTERVAL 5 DAY '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df_increas = panda_df(engine(),sql_now_today)

df_increasA = panda_df(engine(),sql_now_allday)

2023-12-28
(6, 103)
(16, 103)


In [57]:
df_increas = df_increas.sort_values(by=['rate_1'],ascending=[1])
df_increas

,date,code,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,...,rate_91,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100
1,2023-12-28,002011,盾安环境,2.34,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,2023-12-28,000099,中信海直,4.01,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,2023-12-28,603977,国泰集团,4.59,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2023-12-28,603052,可川科技,9.99,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2023-12-28,002962,五方光电,10.00,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2023-12-28,002783,凯龙股份,10.02,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 写入周063

In [58]:
# write_code_to_blk(df_increas.code.tolist(),'063')

### Filter Day多日

In [59]:
df_increas.shape
df_increasA.shape
df_increasA.columns[:10]
# df_iA = df_increasA.groupby(level=[0]).tail(1)
df_iA = groupby_count_code(df_increas)
df_iA_multiIndex = df_iA.reset_index().set_index(['code','date'])
df_iA_multiIndex.shape
# df_iA_multiIndex[:10]
# df_iA_multiIndex[-10:]
# df_iA.loc[df_iA.code == '000028']
df_iA_multiIndex[df_iA_multiIndex.index.get_level_values('code') == '000028'][:2]

(6, 103)

(16, 103)

Index(['date', 'code', 'name', 'rate_1', 'rate_2', 'rate_3', 'rate_4',
       'rate_5', 'rate_6', 'rate_7'],
      dtype='object')

(6, 103)

,,index,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,rate_8,...,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100,couts
code,date,,,,,,,,,,,,,,,,,,,,,


In [60]:
df_iAtail = df_iA_multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df_iAtail =  df_iAtail.reset_index().set_index('code')
df_iAtail.loc[:,['date','name','couts']][:5]
# df_iA.loc[:,['date','code','name','couts']][:50]

# df_iAtail[df_iAtail.couts < 3].loc[:,['date','name','couts']]

,date,name,couts
code,,,
002011,2023-12-28,盾安环境,1
000099,2023-12-28,中信海直,1
603977,2023-12-28,国泰集团,1
603052,2023-12-28,可川科技,1
002962,2023-12-28,五方光电,1


### show Couts

In [61]:
df_iA.loc[:,['date','code','name','couts']]

,date,code,name,couts
1,2023-12-28,002011,盾安环境,1
0,2023-12-28,000099,中信海直,1
5,2023-12-28,603977,国泰集团,1
4,2023-12-28,603052,可川科技,1
3,2023-12-28,002962,五方光电,1
2,2023-12-28,002783,凯龙股份,1


## cn_stock_indicators MutilIndex

### 读取SQL数据表

In [62]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_indicators'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = last_tddate(5)

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(engine(),sql_now_today)

# df2 = panda_df(engine(),sql_now_allday)



# conn.close()

2023-12-28
(27129, 77)


In [63]:
df2.columns
df2.loc[df2.code =='000061'][:5]

Index(['date', 'code', 'name', 'close', 'macd', 'macds', 'macdh', 'kdjk',
       'kdjd', 'kdjj', 'boll_ub', 'boll', 'boll_lb', 'trix', 'trix_20_sma',
       'tema', 'cr', 'cr-ma1', 'cr-ma2', 'cr-ma3', 'rsi_6', 'rsi_12', 'rsi',
       'rsi_24', 'vr', 'vr_6_sma', 'roc', 'rocma', 'rocema', 'pdi', 'mdi',
       'dx', 'adx', 'adxr', 'wr_6', 'wr_10', 'wr_14', 'cci', 'cci_84', 'tr',
       'atr', 'dma', 'dma_10_sma', 'obv', 'sar', 'psy', 'psyma', 'br', 'ar',
       'emv', 'emva', 'bias', 'mfi', 'mfisma', 'vwma', 'mvwma', 'ppo', 'ppos',
       'ppoh', 'wt1', 'wt2', 'supertrend_ub', 'supertrend', 'supertrend_lb',
       'dpo', 'madpo', 'vhf', 'rvi', 'rvis', 'fi', 'force_2', 'force_13',
       'ene_ue', 'ene', 'ene_le', 'stochrsi_k', 'stochrsi_d'],
      dtype='object')

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
43,2023-12-21,000061,农 产 品,6.55,-0.005564,0.030689,-0.036254,25.6037,34.6521,7.50702,...,-0.339950,-0.260194,-255864.0,-260969.0,-113443.0,7.47252,6.79932,6.12612,0.0000,2.605320e-14
4563,2023-12-22,000061,农 产 品,6.49,-0.022779,0.020013,-0.042792,21.7358,30.3466,4.51415,...,-0.351716,-0.308700,-297174.0,-285106.0,-139691.0,7.44699,6.77609,6.10519,0.0000,-1.539510e-14
9083,2023-12-25,000061,农 产 品,6.44,-0.040034,0.007969,-0.048003,19.9451,26.8794,6.07635,...,-0.348093,-0.337689,-232090.0,-249762.0,-152891.0,7.40370,6.73670,6.06970,0.0000,0.000000e+00
13604,2023-12-26,000061,农 产 品,6.36,-0.059447,-0.005503,-0.053943,13.9028,22.5539,-3.39944,...,-0.344140,-0.347285,-328144.0,-302017.0,-177927.0,7.35153,6.68923,6.02693,0.0000,-1.894780e-14
18127,2023-12-27,000061,农 产 品,6.47,-0.065166,-0.017378,-0.047787,16.5412,20.5497,8.52439,...,-0.335006,-0.345198,493724.0,228477.0,-81976.6,7.31601,6.65691,5.99781,32.6186,1.087290e+01


In [64]:
df2[ (df2.close >= df2.boll_ub) & (df2.close * 0.95  <= df2.boll_ub) & (df2.macdh > 0)& (df2.macd > 0)& (df2.macds > 0)]
# .loc[:,['rsi_6','rsi_12','rsi_24','rsi']]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
213,2023-12-21,000677,恒天海龙,3.47,0.034747,0.010968,0.023779,72.4940,63.3175,90.8470,...,0.051318,-0.006328,3785900.0,5784900.0,2368890.0,3.59196,3.26836,2.94476,100.0000,93.3406
225,2023-12-21,000692,*ST惠天,3.06,0.081220,0.059263,0.021958,75.8371,67.0230,93.4652,...,0.170423,0.186698,1175440.0,752833.0,315497.0,3.18903,2.90173,2.61443,97.2347,76.4338
279,2023-12-21,000779,甘咨询,11.61,0.181027,0.051689,0.129338,64.2307,59.2430,74.2059,...,0.154626,0.179615,36936000.0,23324300.0,6372630.0,11.91140,10.83830,9.76521,85.3336,71.7161
300,2023-12-21,000810,创维数字,14.37,0.199663,0.019237,0.180426,74.6565,62.3005,99.3685,...,0.224230,0.095026,-12957200.0,15326800.0,12617000.0,14.57430,13.26130,11.94830,90.1709,96.7236
349,2023-12-21,000890,法尔胜,5.19,0.087674,0.044968,0.042707,67.5994,57.7081,87.3821,...,0.095902,-0.002288,19992400.0,11173500.0,2728370.0,5.30469,4.82679,4.34889,83.8915,81.2331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27031,2023-12-28,605056,咸亨国际,14.17,0.302410,0.280166,0.022244,57.3553,64.0673,43.9315,...,0.039873,0.081736,2762460.0,1888780.0,479938.0,15.26360,13.88850,12.51340,68.8828,48.4247
27050,2023-12-28,605117,德业股份,83.99,3.286900,0.074733,3.212170,88.3801,80.9387,103.2630,...,0.511216,0.391345,103709000.0,81705000.0,59146000.0,77.73000,70.72730,63.72460,100.0000,99.3212
27067,2023-12-28,605178,时空科技,25.21,0.418221,0.273537,0.144684,73.6920,57.1365,106.8030,...,0.028073,-0.039605,19274000.0,13050700.0,3122220.0,25.10040,22.83910,20.57780,100.0000,68.5599
27074,2023-12-28,605196,华通线缆,7.72,0.031108,0.008106,0.023003,69.3379,63.1123,81.7890,...,0.139959,0.159707,1874080.0,1763420.0,541018.0,8.36496,7.61136,6.85776,100.0000,75.3098


# inStock 序列化数据测试

## 定义参数

In [65]:
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\2020'+get_today(sep='')[-4:-2]+'\\2020'+get_today(sep='')[-4:]
stock_hist_cache_path

'Y:\\dockerf\\stock\\instockcache\\hist\\202012\\20201228'

In [66]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
get_today()
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\2020'+get_today(sep='')[-4:]
code = '601336'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


'2023-12-28'

Y:\dockerf\stock\instockcache\hist\202012\20201228\601336qfq.gzip.pickle


In [67]:
df.shape
df[-2:]
# df.columns
lastday=df.date.values[-1]
lastday
data = df.copy()
# macd
data.loc[:, 'diff'], data.loc[:, 'dea'], data.loc[:, 'macd'] = tl.MACD(
    data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['diff'].values[np.isnan(data['diff'].values)] = 0.0
data['dea'].values[np.isnan(data['dea'].values)] = 0.0
data['macd'].values[np.isnan(data['macd'].values)] = 0.0

data.loc[:, 'boll_ub'], data.loc[:, 'boll'], data.loc[:, 'boll_lb'] = tl.BBANDS \
    (data['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['boll_ub'].values[np.isnan(data['boll_ub'].values)] = 0.0
data['boll'].values[np.isnan(data['boll'].values)] = 0.0
data['boll_lb'].values[np.isnan(data['boll_lb'].values)] = 0.0
data.iloc[-60:]['macd'].max()
# DIFF:=EMA(CLOSE,12) - EMA(CLOSE,26);  256 
# DEA := EMA(DIFF,9); 1.52
# MACD := 2*(DIFF-DEA); 2.08
print(not False)

(730, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
728,2023-12-27,29.50,29.34,29.68,29.05,75994,222485775.0,2.14,-0.54,-0.16,0.36
729,2023-12-28,29.25,31.10,31.12,29.21,194483,591652698.0,6.51,6.00,1.76,0.93


'2023-12-28'

0.18143832828130413

True


## stock_period_to_typ Resample 'W-FRI'

In [68]:
def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).first()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()
    
    lastWeek1 = str(period_stock_data['open'].index.values[-1])[:10]
    lastweek2 = str(period_stock_data['open'].index.values[-2])[:10]
    if ratiodays:
        if period_day == 'W-FRI':
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
            ratio_d =(5-(duratio%5))/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day.find('W') >= 0:
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastday, '%Y-%m-%d').date() - datetime.datetime.strptime(lastweek2, '%Y-%m-%d').date())[0])
            ratio_d =(duratio)/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day == 'BM':
            # daynow = '2023-04-26'
            # lastday = '2023-04-23'
            print(lastWeek1,lastweek2,lastday,lastday2)
            # print((str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2]))
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2])
            ratio_d =(30-(duratio%30))/30
            print("ratio_d:%s %s dura:%s"%(ratio_d,lastday,duratio))
        elif period_day.find('M') >= 0:
            ratio_d = 1
            
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays and ratio_d != 1:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    if not indextype and period_stock_data.index.name == 'date':
        # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
        period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
        period_stock_data.index.name = 'date'
    else:
        if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    # print period_stock_data
    return period_stock_data


### Resample Week W-FRI

In [69]:
# dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
# dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
# data[-5:]

2023-12-29 2023-12-22 2023-12-28 2023-12-27
ratio_d:0.8 2023-12-28


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-12-15,30.15,30.12,31.02,29.58,503832.5,1.531563e+09,16.1,-0.1,0.04,2.4,-0.839538,-0.900252,0.060714,32.730483,31.1140,29.497517,2023-12-15
2023-12-22,29.94,29.42,30.17,28.98,479253.8,1.416087e+09,13.1,-2.9,-0.02,2.3,-0.861634,-0.873382,0.011747,31.781969,30.4225,29.063031,2023-12-22
2023-12-29,29.30,31.10,31.12,29.05,497043.8,1.484187e+09,14.8,7.2,1.76,2.4,-0.628411,-0.801224,0.172813,31.450182,30.1430,28.835818,2023-12-28


2023-12-31 2023-12-24 2023-12-28 2023-12-27
ratio_d:0.8 2023-12-28


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-12-17,30.15,30.12,31.02,29.58,503832.5,1.531563e+09,16.1,-0.1,0.04,2.4,-0.839538,-0.900252,0.060714,32.730483,31.1140,29.497517,2023-12-15
2023-12-24,29.94,29.42,30.17,28.98,479253.8,1.416087e+09,13.1,-2.9,-0.02,2.3,-0.861634,-0.873382,0.011747,31.781969,30.4225,29.063031,2023-12-22
2023-12-31,29.30,31.10,31.12,29.05,497043.8,1.484187e+09,14.8,7.2,1.76,2.4,-0.628411,-0.801224,0.172813,31.450182,30.1430,28.835818,2023-12-28


### Resample Month BM
https://note.nkmk.me/python-pandas-time-series-freq/

    BM: 月末最終営業日ごと
    MS: 毎月の開始日ごと
    BMS: 毎月の最初の営業日ごと


In [70]:
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-2:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]

1


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-11-30,33.66,31.25,33.72,30.80,2401477,7.755185e+09,45.92,-6.46,0.18,11.51,-0.962788,-1.020240,0.057452,33.410008,32.208,31.005992,2023-11-30
2023-12-29,31.11,31.10,31.70,28.98,1668663,5.032113e+09,47.67,-0.18,1.76,8.00,-0.628411,-0.801224,0.172813,31.450182,30.143,28.835818,2023-12-28


2023-12-29 2023-11-30 2023-12-28 2023-12-27
ratio_d:0.9666666666666667 2023-12-28 dura:1


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-10-31,36.60,33.40,38.10,32.33,2183464.1,7.776926e+09,48.6,-9.9,0.05,10.5,-1.275983,-1.098621,-0.177362,38.396352,35.978,33.559648,2023-10-31
2023-11-30,33.66,31.25,33.72,30.80,2484286.6,8.022605e+09,47.5,-6.7,0.18,11.9,-0.962788,-1.020240,0.057452,33.410008,32.208,31.005992,2023-11-30
2023-12-29,31.11,31.10,31.70,28.98,1726203.1,5.205634e+09,49.3,-0.2,1.76,8.3,-0.628411,-0.801224,0.172813,31.450182,30.143,28.835818,2023-12-28


In [71]:
daynow=dfw.index.values[-1]
str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


'1'

1

# inStock RSI

In [72]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")

Y:\dockerf\stock\instockcache\hist\202012\20201228\000002qfq.gzip.pickle


In [73]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday

(730, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
725,2023-12-22,10.40,10.38,10.50,10.31,470699,4.891901e+08,1.82,-0.38,-0.04,0.48
726,2023-12-25,10.34,10.30,10.38,10.28,273205,2.817907e+08,0.96,-0.77,-0.08,0.28
727,2023-12-26,10.30,10.17,10.33,10.14,351023,3.573613e+08,1.84,-1.26,-0.13,0.36
728,2023-12-27,10.17,10.17,10.25,10.09,343638,3.489655e+08,1.57,0.00,0.00,0.35
729,2023-12-28,10.15,10.52,10.56,10.10,789808,8.216411e+08,4.52,3.44,0.35,0.81


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
0,2020-12-28,25.27,25.51,25.74,24.95,583681,1.656263e+09,3.14,1.27,0.32,0.6


'2023-12-28'

In [74]:
data=df.copy()

In [75]:
def check_rsi_status(df,threshold=60,period_day=False):
    # macd
    data = df.copy()
    # cci 计算方法和结果和stockstats不同，stockstats典型价采用均价(总额/成交量)计算
    data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
    data['cci'].values[np.isnan(data['cci'].values)] = 0.0
    # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
    # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
    rsilast5 = data.iloc[-6:-1]['cci']
    rsilast5max = rsilast5.max()
    rsilast = data.iloc[-1]['cci']

    if rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max:
        return True
    else:
        return False

In [76]:
# data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
# data['cci'].values[np.isnan(data['cci'].values)] = 0.0
# # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
# # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
# rsilast5 = data.iloc[-6:-1]['cci']
# rsilast5max = rsilast5.max()
# rsilast = data.iloc[-1]['cci']
# data['cci'][-6:]
# rsilast5.iloc[-1]
# rsilast5,rsilast
# rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max
check_rsi_status(df)

True